In [1]:
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import DateOffset
from networkdays import networkdays
import numpy as np
from pandas.tseries.offsets import BDay
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def bond_flows(start,bond_type, end,holiday,vna_ntnb,pu_intra):
    start = datetime.strptime(start , '%d/%m/%Y')
    end = datetime.strptime(end, '%d/%m/%Y')
    df = pd.DataFrame(columns = ["Data","TP","Vencimento","Data Fluxo", "Tipo", "du", "Taxa", "VNA", "Taxa Intra", "PU Intra"])
    if bond_type == 'NTN-B':
        months = (end.year - start.year)*12 + end.month - start.month
        period = 6
        for month in range(0, months, period):
            date = (end - DateOffset(months=month)) + BDay(0)
            days = networkdays.Networkdays(start,date,holidays=holiday)
            du = len(days.networkdays())-1
            taxa_cupom = ((1+0.06)**(1/2)-1)*100
            vna = get_vna(start.strftime('%Y-%m-%d'),bond_type,vna_ntnb)
            pu = is_pu(start.strftime('%Y-%m-%d'),bond_type,pu_intra,end.strftime('%Y-%m-%d'),'True')
            taxa = is_pu(start.strftime('%Y-%m-%d'),bond_type,pu_intra,end.strftime('%Y-%m-%d'),'False')
            if month ==0:
                dict = {'Data Fluxo': date.strftime('%Y-%m-%d') , 'Tipo': 'Amortização', 'du': du , 'VNA': vna, 'Taxa Intra': taxa, 'PU Intra': pu,  'Taxa': 100 , 'TP': bond_type, 'Vencimento': end , 'Data': start}
                df = df.append(dict, ignore_index = True)
                dict = {'Data Fluxo': date.strftime('%Y-%m-%d') , 'Tipo': 'Juros', 'du': du, 'VNA': vna, 'Taxa Intra': taxa, 'PU Intra': pu, 'Taxa': taxa_cupom, 'TP': bond_type, 'Vencimento': end, 'Data': start}
                df = df.append(dict, ignore_index = True)
            else:
                dict = {'Data Fluxo': date.strftime('%Y-%m-%d') , 'Tipo': 'Juros', 'du': du, 'VNA': vna, 'Taxa Intra': taxa, 'PU Intra': pu, 'Taxa': taxa_cupom, 'TP': bond_type, 'Vencimento': end, 'Data': start}
                df = df.append(dict, ignore_index = True)   
    elif bond_type =='NTN-B Princ':
        print('fazer')
    else:
        print('Escolher NTN-B ou NTN-B Princ')
    df = df.sort_values(by="Data Fluxo")
    return df

def get_vna(data,bond_type,vna_ntnb):
    vna_filter = vna_ntnb[(vna_ntnb['data_referencia']==data) & (vna_ntnb['Tipo']==bond_type)]
    return vna_filter['vna'].iloc[0]


def is_pu(data,bond_type,pu,end,bool_pu):
    pu = pu[(pu['data_referencia']==data) & (pu['Tipo']==bond_type) & (pu['data_vencimento']==end)]
    if bool_pu == 'True':
        value = pu['pu'].iloc[0]
    else:
        value = pu['taxa_intradiaria'].iloc[0]
    return value


def joao_completo(df1):
    df = df1.copy()
    df['Valor Futuro'] = df['VNA']*df['Taxa']/100
    df['Valor Presente'] = df['Valor Futuro']/((1+df['Taxa Intra']/100)**(df['du']/252))
    df['PU Calc'] = df['Valor Presente'].sum()
    df['Dur Parcial'] = (df['Valor Presente']*df['du']/df['PU Calc'])
    df['Duration'] = df['Dur Parcial'].sum()/252
    df['Duration Modificada'] = df['Duration']/(1+df['Taxa Intra']/100)
    df['DV01'] = df['PU Calc']*(df['Duration Modificada'])*(0.01/100)
    df['DV10'] = df['PU Calc']*(df['Duration Modificada'])*(0.1/100)
    return df
    
    
def taxa_ettj(data,dados_curva,du):
   
    parametros = dados_curva.query("grupo_indexador=='PREFIXADOS' & data_referencia=='"+str(data +"'"))

    data_ref = parametros['data_referencia'].iloc[0]
    b1 = parametros['b1'].iloc[0]
    b2 = parametros['b2'].iloc[0]
    b3 = parametros['b3'].iloc[0]
    b4 = parametros['b4'].iloc[0]
    l1 = parametros['l1'].iloc[0]
    l2 = parametros['l2'].iloc[0]
    tau = du/252
    juros_pre = b1 + b2*( (1 - np.exp(-l1*tau)) / (l1*tau) ) + b3*( (1 - np.exp(-l1*tau)) / (l1*tau) - np.exp(-l1*tau) ) + b4*( (1 - np.exp(-l2*tau)) / (l2*tau) - np.exp(-l2*tau) )
    return juros_pre*100

In [3]:
dados_curva = pd.read_csv('curva1.csv')
titulos = pd.read_csv('titulos.csv')
feriados = pd.read_excel('feriados_nacionais.xls', header =0, sheet_name="Plan1").dropna()
feriados_list=feriados['Data'].to_list()
vna = pd.read_csv('vna.csv')
vna_ntnb = vna[vna['tipo_titulo']=='NTN-B']
#vna_ntnb['Tipo'] = vna_ntnb['codigo_selic'].apply(lambda x: 'NTN-B' if x == 760199 else 'NTN-B Princ')
vna_ntnb.loc[vna_ntnb['codigo_selic'] == 760199, 'Tipo'] = 'NTN-B' 
vna_ntnb.loc[vna_ntnb['codigo_selic'] != 760199, 'Tipo'] = 'NTN-B Princ' 
pu = pd.read_csv('precos.csv')
pu.loc[pu['codigo_selic'] == 760199, 'Tipo'] = 'NTN-B' 
pu.loc[pu['codigo_selic'] != 760199, 'Tipo'] = 'NTN-B Princ' 

In [4]:
ntnb_list = titulos[titulos['Tipo']=='NTN-B']
ntnb_list

,Unnamed: 0,Vencimento,Emissor,Tipo,SELIC_ID
25,25,08/2022,Tesouro Nacional,NTN-B,76019920220815
26,26,03/2023,Tesouro Nacional,NTN-B,76010020230315
27,27,05/2023,Tesouro Nacional,NTN-B,76019920230515
28,28,08/2024,Tesouro Nacional,NTN-B,76019920240815
29,29,05/2025,Tesouro Nacional,NTN-B,76019920250515
30,30,08/2026,Tesouro Nacional,NTN-B,76019920260815
31,31,05/2027,Tesouro Nacional,NTN-B,76019920270515
32,32,08/2028,Tesouro Nacional,NTN-B,76019920280815
33,33,08/2030,Tesouro Nacional,NTN-B,76019920300815
34,34,08/2032,Tesouro Nacional,NTN-B,76019920320815


In [5]:
data_do_calculo = '13/05/2022' #as datas dos calculos são de 01/01/2022 até 13/05/2022
bond_type = 'NTN-B' #colocar só NTN-B
vencimento = '15/08/2026'  #vencimento da ntn_b
flows = bond_flows(data_do_calculo,bond_type, vencimento,feriados_list,vna_ntnb,pu)
completo = joao_completo(flows)

In [6]:
from IPython.display import Markdown as md
titulo = completo['TP'].iloc[0]
vencimento=completo['Vencimento'].iloc[0]
data =completo['Data'].iloc[0]
taxa =completo['Taxa Intra'].iloc[0]
vna_cal =completo['VNA'].iloc[0]
pu_intra =completo['PU Intra'].iloc[0]
pu_calc = completo['PU Calc'].iloc[0]
pu_dif = pu_intra - pu_calc
duration = completo['Duration'].iloc[0]
duration_mod = completo['Duration Modificada'].iloc[0]
dv_01 = completo['DV01'].iloc[0]
dv_10 = completo['DV10'].iloc[0]
fluxos = completo[['Data','TP','Vencimento','Data Fluxo','Tipo','du','Taxa','Valor Futuro','Valor Presente','Dur Parcial']].to_markdown()
md("""
<h3>Resultados</h3>
Título: {} <br/>
   Vencimento: {} <br/>
   Data do Cálculo: {} <br/>
   Taxa intradiária: {}  <br/>
   VNA: {}  <br/>
   PU Intradiaria: {}  <br/>
   PU Calculado: {}  <br/>
   PU Intradiaria - PU Calculado: {}  <br/>
   Duration: {}  <br/>
   Duration Modificada: {}  <br/>
   DV01: {}  <br/>
   DV10: {}  <br/>
   """.format(titulo,vencimento,data,taxa,vna_cal,pu_intra,pu_calc,pu_dif,duration,duration_mod,dv_01,dv_10))


<h3>Resultados</h3>
Título: NTN-B <br/>
   Vencimento: 2026-08-15 00:00:00 <br/>
   Data do Cálculo: 2022-05-13 00:00:00 <br/>
   Taxa intradiária: 5.5434  <br/>
   VNA: 3950.984558  <br/>
   PU Intradiaria: 4073.74955  <br/>
   PU Calculado: 4073.7739901090386  <br/>
   PU Intradiaria - PU Calculado: -0.024440109038550872  <br/>
   Duration: 3.7756680345617517  <br/>
   Duration Modificada: 3.5773606256400226  <br/>
   DV01: 1.4573358669972523  <br/>
   DV10: 14.573358669972523  <br/>
   

In [7]:
md(completo[['Data','TP','Vencimento','Data Fluxo','Tipo','du','Taxa','Valor Futuro','Valor Presente','Dur Parcial']].to_markdown())

|    | Data                | TP    | Vencimento          | Data Fluxo   | Tipo        |   du |     Taxa |   Valor Futuro |   Valor Presente |   Dur Parcial |
|---:|:--------------------|:------|:--------------------|:-------------|:------------|-----:|---------:|---------------:|-----------------:|--------------:|
|  9 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2022-08-15   | Juros       |   65 |   2.9563 |        116.803 |         115.189  |       1.83792 |
|  8 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2023-02-15   | Juros       |  193 |   2.9563 |        116.803 |         112.075  |       5.30969 |
|  7 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2023-08-15   | Juros       |  316 |   2.9563 |        116.803 |         109.162  |       8.46764 |
|  6 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2024-02-15   | Juros       |  440 |   2.9563 |        116.803 |         106.302  |      11.4815  |
|  5 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2024-08-15   | Juros       |  567 |   2.9563 |        116.803 |         103.451  |      14.3986  |
|  4 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2025-02-17   | Juros       |  696 |   2.9563 |        116.803 |         100.633  |      17.193   |
|  3 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2025-08-15   | Juros       |  819 |   2.9563 |        116.803 |          98.0174 |      19.7056  |
|  2 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2026-02-16   | Juros       |  947 |   2.9563 |        116.803 |          95.3677 |      22.1694  |
|  0 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2026-08-17   | Amortização | 1072 | 100      |       3950.98  |        3140.73   |     826.472   |
|  1 | 2022-05-13 00:00:00 | NTN-B | 2026-08-15 00:00:00 | 2026-08-17   | Juros       | 1072 |   2.9563 |        116.803 |          92.8494 |      24.433   |